In [5]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import spearmanr
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

# Prepare features and target - FIXED
X = df.loc[:, 'BIC_TAI':'BRD_Avg_Echo']  # First 9 columns as features (actual data, not column names)
y = df['Brooke']
subjects = df['subject']

# Remove rows where subject or target is NaN
valid_mask = ~(subjects.isna() | y.isna())
X = X[valid_mask]
y = y[valid_mask]
subjects = subjects[valid_mask]

print(f"After removing NaN values:")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"subjects shape: {subjects.shape}")

# Initialize models with 20 core CPU utilization
models = {
    'KNN': KNeighborsRegressor(n_neighbors=5, n_jobs=20),
    'SVM': SVR(kernel='rbf', C=1.0, gamma='scale'),  # SVM doesn't support n_jobs
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=20),
    'XGBoost': xgb.XGBRegressor(random_state=42, verbosity=0, n_jobs=20),
    'Ridge': Ridge(alpha=1.0),  # Ridge doesn't support n_jobs
    'Lasso': Lasso(alpha=0.1, max_iter=1000),  # Lasso doesn't support n_jobs
    'MLP': MLPRegressor(
        hidden_layer_sizes=(100, 50),  # Two hidden layers with 100 and 50 neurons
        activation='relu',
        solver='adam',
        alpha=0.001,  # L2 regularization
        batch_size='auto',
        learning_rate='constant',
        learning_rate_init=0.001,
        max_iter=1000,
        random_state=42,
        early_stopping=True,
        validation_fraction=0.1,
        n_iter_no_change=10,
        tol=1e-4
    )
}

# Function to calculate performance metrics
def calculate_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    
    # Spearman correlation with p-value
    correlation, p_value = spearmanr(y_true, y_pred)
    
    return mae, mse, correlation, p_value

# Leave-One-Subject-Out Cross-Validation with Combined Predictions
def loso_cv_combined(X, y, subjects, models):
    unique_subjects = subjects.dropna().unique()
    
    # Store all predictions and true values for each model
    all_predictions = {model_name: {'y_true': [], 'y_pred': []} 
                      for model_name in models.keys()}
    
    print(f"Performing Leave-One-Subject-Out CV with {len(unique_subjects)} subjects...")
    
    for i, test_subject in enumerate(unique_subjects):
        print(f"Processing subject {test_subject} ({i+1}/{len(unique_subjects)})")
        
        # Split data
        train_mask = subjects != test_subject
        test_mask = subjects == test_subject
        
        X_train, X_test = X[train_mask], X[test_mask]
        y_train, y_test = y[train_mask], y[test_mask]
        
        # Skip if test set is too small
        if len(y_test) < 1:
            print(f"  Skipping subject {test_subject} - insufficient data ({len(y_test)} samples)")
            continue
            
        # Standardize features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # Train and evaluate each model
        for model_name, model in models.items():
            try:
                # Train model
                model.fit(X_train_scaled, y_train)
                
                # Make predictions
                y_pred = model.predict(X_test_scaled)
                
                # Store predictions and true values
                all_predictions[model_name]['y_true'].extend(y_test.tolist())
                all_predictions[model_name]['y_pred'].extend(y_pred.tolist())
                
            except Exception as e:
                print(f"  Error with {model_name} for subject {test_subject}: {e}")
                continue
    
    # Calculate overall performance metrics for each model
    results = {}
    for model_name in models.keys():
        if len(all_predictions[model_name]['y_true']) > 0:
            y_true_all = np.array(all_predictions[model_name]['y_true'])
            y_pred_all = np.array(all_predictions[model_name]['y_pred'])
            
            mae, mse, correlation, p_value = calculate_metrics(y_true_all, y_pred_all)
            
            results[model_name] = {
                'mae': mae,
                'mse': mse,
                'correlation': correlation,
                'p_value': p_value,
                'n_samples': len(y_true_all),
                'y_true': y_true_all,
                'y_pred': y_pred_all
            }
    
    return results

# Run LOSO CV with combined predictions
results = loso_cv_combined(X, y, subjects, models)

# Create summary DataFrame
def create_summary_df(results):
    summary_data = []
    
    for model_name, metrics in results.items():
        summary_data.append({
            'Model': model_name,
            'MAE': metrics['mae'],
            'MSE': metrics['mse'],
            'RMSE': np.sqrt(metrics['mse']),
            'Correlation': metrics['correlation'],
            'P_value': metrics['p_value'],
            'N_samples': metrics['n_samples']
        })
    
    return pd.DataFrame(summary_data)

# Generate summary
summary_df = create_summary_df(results)

# Display results
print("\n" + "="*80)
print("LEAVE-ONE-SUBJECT-OUT CROSS-VALIDATION RESULTS")
print("(Combined predictions across all subjects)")
print("="*80)

print(f"\nDataset Info:")
print(f"- Total samples: {len(X)}")
print(f"- Features: {X.shape[1]}")
print(f"- Subjects: {len(subjects.unique())}")
print(f"- Target variable: Brooke")

print(f"\nOverall Performance (All Predictions Combined):")
print(summary_df.round(4))

# Detailed results for each model
print("\n" + "="*80)
print("DETAILED RESULTS BY MODEL")
print("="*80)

for model_name in models.keys():
    if model_name in results:
        print(f"\n{model_name}:")
        print(f"  MAE: {results[model_name]['mae']:.4f}")
        print(f"  MSE: {results[model_name]['mse']:.4f}")
        print(f"  RMSE: {np.sqrt(results[model_name]['mse']):.4f}")
        print(f"  Spearman Correlation: {results[model_name]['correlation']:.4f}")
        print(f"  P-value: {results[model_name]['p_value']:.4f}")
        print(f"  Total predictions: {results[model_name]['n_samples']}")

# Find best model for each metric
print("\n" + "="*80)
print("BEST MODELS BY METRIC")
print("="*80)

if not summary_df.empty:
    best_mae = summary_df.loc[summary_df['MAE'].idxmin(), 'Model']
    best_mse = summary_df.loc[summary_df['MSE'].idxmin(), 'Model']
    best_rmse = summary_df.loc[summary_df['RMSE'].idxmin(), 'Model']
    best_corr = summary_df.loc[summary_df['Correlation'].idxmax(), 'Model']
    
    print(f"Best MAE: {best_mae} ({summary_df.loc[summary_df['MAE'].idxmin(), 'MAE']:.4f})")
    print(f"Best MSE: {best_mse} ({summary_df.loc[summary_df['MSE'].idxmin(), 'MSE']:.4f})")
    print(f"Best RMSE: {best_rmse} ({summary_df.loc[summary_df['RMSE'].idxmin(), 'RMSE']:.4f})")
    print(f"Best Correlation: {best_corr} ({summary_df.loc[summary_df['Correlation'].idxmax(), 'Correlation']:.4f})")

# Statistical significance analysis
print("\n" + "="*80)
print("STATISTICAL SIGNIFICANCE ANALYSIS")
print("="*80)

for model_name in models.keys():
    if model_name in results:
        p_value = results[model_name]['p_value']
        significance = "significant" if p_value < 0.05 else "not significant"
        print(f"{model_name}: p = {p_value:.4f} ({significance})")

print("\nAnalysis complete!")

# Optional: Save predictions for further analysis
print("\n" + "="*80)
print("PREDICTION STORAGE")
print("="*80)

print("All predictions are stored in the 'results' dictionary.")
print("Access them using: results['ModelName']['y_true'] and results['ModelName']['y_pred']")
print("Example: results['Random Forest']['y_true'] contains all true values")
print("Example: results['Random Forest']['y_pred'] contains all predicted values")

# MLP specific information
print("\n" + "="*80)
print("MLP MODEL CONFIGURATION")
print("="*80)

print("MLP Configuration:")
print("- Hidden layers: (100, 50) - Two hidden layers with 100 and 50 neurons")
print("- Activation: ReLU")
print("- Solver: Adam optimizer")
print("- Learning rate: 0.001")
print("- L2 regularization (alpha): 0.001")
print("- Early stopping: Enabled")
print("- Max iterations: 1000")
print("- Tolerance: 1e-4")
print("\nNote: MLP may take longer to train compared to other models.")

After removing NaN values:
X shape: (99, 12)
y shape: (99,)
subjects shape: (99,)
Performing Leave-One-Subject-Out CV with 45 subjects...
Processing subject 1 (1/45)
Processing subject 2 (2/45)
Processing subject 3 (3/45)
Processing subject 4 (4/45)
Processing subject 5 (5/45)
Processing subject 6 (6/45)
Processing subject 7 (7/45)
Processing subject 8 (8/45)
Processing subject 9 (9/45)
Processing subject 10 (10/45)
Processing subject 11 (11/45)
Processing subject 14 (12/45)
Processing subject 16 (13/45)
Processing subject 17 (14/45)
Processing subject 18 (15/45)
Processing subject 19 (16/45)
Processing subject 20 (17/45)
Processing subject 21 (18/45)
Processing subject 22 (19/45)
Processing subject 23 (20/45)
Processing subject 24 (21/45)
Processing subject 25 (22/45)
Processing subject 26 (23/45)
Processing subject 27 (24/45)
Processing subject 28 (25/45)
Processing subject 30 (26/45)
Processing subject 31 (27/45)
Processing subject 32 (28/45)
Processing subject 33 (29/45)
Processing

In [10]:
# summary_df.drop(columns=['N_samples'])

In [14]:
summary_df

,Model,RMSE,NRMSE,Correlation,P_value
0,KNN,0.771722,0.192931,0.404158,0.000033
1,SVM,0.756215,0.189054,0.439039,0.000005
2,Random Forest,0.777529,0.194382,0.415467,0.000019
3,XGBoost,0.941386,0.235346,0.286116,0.004093
4,Ridge,0.777460,0.194365,0.395268,0.000051
5,Lasso,0.736283,0.184071,0.455537,0.000002
6,MLP,0.820824,0.205206,0.419495,0.000015


In [13]:
# summary_df = summary_df.drop(columns=['R²'])

In [15]:
# results

In [16]:
# results['KNN']

In [7]:
import pandas as pd
from sklearn.metrics import r2_score

# Calculate R² for each model
r2_scores = []
for model_name in results.keys():
    y_true = results[model_name]['y_true']
    y_pred = results[model_name]['y_pred']
    r2 = r2_score(y_true, y_pred)
    r2_scores.append(r2)

# Add R² column to summary_df
summary_df['R²'] = r2_scores

# Drop N_samples column
summary_df = summary_df.drop('N_samples', axis=1)

# Reorder columns for better presentation
summary_df = summary_df[['Model', 'MAE', 'MSE', 'RMSE', 'R²', 'Correlation', 'P_value']]

print("Updated summary with R² scores:")
print(summary_df)

# Optional: Sort by R² (descending) to see best performing models
print("\nSummary sorted by R² (best to worst):")
summary_sorted = summary_df.sort_values('R²', ascending=False).reset_index(drop=True)
summary_sorted

Updated summary with R² scores:
           Model       MAE       MSE      RMSE        R²  Correlation  \
0            KNN  0.315152  0.595556  0.771722  0.128540     0.392930   
1            SVM  0.368557  0.571861  0.756215  0.163212     0.400054   
2  Random Forest  0.371111  0.604552  0.777529  0.115376     0.464783   
3        XGBoost  0.389412  0.886207  0.941386 -0.296763     0.447969   
4          Ridge  0.465706  0.604444  0.777460  0.115533     0.460491   
5          Lasso  0.411687  0.542112  0.736283  0.206742     0.498056   
6            MLP  0.517555  0.673752  0.820824  0.014117     0.116343   

        P_value  
0  5.745788e-05  
1  4.084738e-05  
2  1.256609e-06  
3  3.328308e-06  
4  1.619312e-06  
5  1.558167e-07  
6  2.514674e-01  

Summary sorted by R² (best to worst):


,Model,MAE,MSE,RMSE,R²,Correlation,P_value
0,Lasso,0.411687,0.542112,0.736283,0.206742,0.498056,1.558167e-07
1,SVM,0.368557,0.571861,0.756215,0.163212,0.400054,4.084738e-05
2,KNN,0.315152,0.595556,0.771722,0.128540,0.392930,5.745788e-05
3,Ridge,0.465706,0.604444,0.777460,0.115533,0.460491,1.619312e-06
4,Random Forest,0.371111,0.604552,0.777529,0.115376,0.464783,1.256609e-06
5,MLP,0.517555,0.673752,0.820824,0.014117,0.116343,2.514674e-01
6,XGBoost,0.389412,0.886207,0.941386,-0.296763,0.447969,3.328308e-06


In [7]:
summary_sorted.to_csv('results.csv', index=False)

In [8]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from scipy.stats import pearsonr

# Calculate the required metrics for each model
summary_data = []

for model_name in results.keys():
    y_true = results[model_name]['y_true']
    y_pred = results[model_name]['y_pred']
    
    # Calculate RMSE
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    
    # Calculate NRMSE (Normalized RMSE)
    # Using range normalization: NRMSE = RMSE / (max(y_true) - min(y_true))
    nrmse = rmse / (np.max(y_true) - np.min(y_true))
    
    # Calculate R²
    r2 = r2_score(y_true, y_pred)
    
    # Calculate Correlation and p-value
    correlation, p_value = pearsonr(y_true, y_pred)
    
    # Store results
    summary_data.append({
        'Model': model_name,
        'RMSE': rmse,
        'NRMSE': nrmse,
        'R²': r2,
        'Correlation': correlation,
        'P_value': p_value
    })

# Create new summary dataframe
summary_df = pd.DataFrame(summary_data)

print("Summary with selected metrics:")
print(summary_df)

# Optional: Sort by R² (descending) to see best performing models
print("\nSummary sorted by R² (best to worst):")
summary_sorted = summary_df.sort_values('R²', ascending=False).reset_index(drop=True)
print(summary_sorted)

# Optional: Display with formatted numbers for better readability
print("\nFormatted summary:")
summary_formatted = summary_df.copy()
summary_formatted['RMSE'] = summary_formatted['RMSE'].round(6)
summary_formatted['NRMSE'] = summary_formatted['NRMSE'].round(6)
summary_formatted['R²'] = summary_formatted['R²'].round(6)
summary_formatted['Correlation'] = summary_formatted['Correlation'].round(6)
summary_formatted['P_value'] = summary_formatted['P_value'].round(6)
summary_formatted

Summary with selected metrics:
           Model      RMSE     NRMSE        R²  Correlation   P_value
0            KNN  0.771722  0.192931  0.128540     0.404158  0.000033
1            SVM  0.756215  0.189054  0.163212     0.439039  0.000005
2  Random Forest  0.777529  0.194382  0.115376     0.415467  0.000019
3        XGBoost  0.941386  0.235346 -0.296763     0.286116  0.004093
4          Ridge  0.777460  0.194365  0.115533     0.395268  0.000051
5          Lasso  0.736283  0.184071  0.206742     0.455537  0.000002
6            MLP  0.820824  0.205206  0.014117     0.419495  0.000015

Summary sorted by R² (best to worst):
           Model      RMSE     NRMSE        R²  Correlation   P_value
0          Lasso  0.736283  0.184071  0.206742     0.455537  0.000002
1            SVM  0.756215  0.189054  0.163212     0.439039  0.000005
2            KNN  0.771722  0.192931  0.128540     0.404158  0.000033
3          Ridge  0.777460  0.194365  0.115533     0.395268  0.000051
4  Random Forest  0.

,Model,RMSE,NRMSE,R²,Correlation,P_value
0,KNN,0.771722,0.192931,0.128540,0.404158,0.000033
1,SVM,0.756215,0.189054,0.163212,0.439039,0.000005
2,Random Forest,0.777529,0.194382,0.115376,0.415467,0.000019
3,XGBoost,0.941386,0.235346,-0.296763,0.286116,0.004093
4,Ridge,0.777460,0.194365,0.115533,0.395268,0.000051
5,Lasso,0.736283,0.184071,0.206742,0.455537,0.000002
6,MLP,0.820824,0.205206,0.014117,0.419495,0.000015


In [10]:
summary_formatted.to_csv('results.csv',index=False)

In [11]:
df = pd.read_csv('results.csv')
results_df = df

In [12]:
results_df

,Model,RMSE,NRMSE,R²,Correlation,P_value
0,KNN,1.086986,0.271746,0.279134,0.553435,0.003359
1,SVM,1.173204,0.293301,0.160243,0.463342,0.017130
2,Random Forest,1.001353,0.250338,0.388240,0.625525,0.000632
3,XGBoost,0.999949,0.249987,0.389954,0.640226,0.000427
4,Ridge,1.168201,0.292050,0.167390,0.473580,0.014534
5,Lasso,1.161626,0.290407,0.176735,0.445151,0.022677
6,MLP,1.751996,0.437999,0.127272,0.220872,0.278231
